In [187]:
from http.client import REQUESTED_RANGE_NOT_SATISFIABLE
import pandas as pd
from sodapy import Socrata
import matplotlib.pyplot as plt
import geo_comm_areas as gca
pd.set_option('display.max_rows', None)


/home/anth0nyhak1m/.local/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
id_dict = {
    "red light violations": "spqx-js37", "abandoned buildings": "kc9i-wq85",
    "socioeconomic indicators": "kn9c-c2s2", "hardship index": "792q-4jtu",
    "crimes": "ijzp-q8t2", "grocery stores": "4u6w-irs9" }



In [3]:
def pull_data(dataset_id, lim):
    '''
    Given a dataset id, pull out data from Chicago portal API

    Inputs:
      dataset_id: (str) a unique id for one Chicago portal dataset
      limit: (int) number of rows to pull each time, default to None

    Return:
      (Pandas DataFrame) a dataframe pulled from Chicago Portal
    '''
    client = Socrata("data.cityofchicago.org", "goD601SLndI51xcMq1KsnG6np")

    results = client.get(dataset_id, limit = lim)
    return pd.DataFrame.from_records(results)

In [4]:
def gen_tables(id_dictionary, lim):
    dataset_dct = {}
    for dataset_name, url in id_dictionary.items():
        dataset_dct[dataset_name] = pull_data(url, lim)

    return dataset_dct

In [ ]:
dct = gen_tables(id_dict, 1000)



In [ ]:
for key in dct.keys():
    print(key)

In [117]:


client = Socrata("data.cityofchicago.org", "goD601SLndI51xcMq1KsnG6np")
results = client.get("ijzp-q8t2", where = f"(primary_type) = 'HOMICIDE'",
                    limit=1000000)

crime_df = pd.DataFrame.from_records(results)


In [ ]:

crime_df.columns.unique()

In [138]:
crime_year_count = crime_df.groupby(by=['community_area', "year"]).size().unstack()




In [206]:
crime_year_count = crime_year_count.fillna(0)

crime_year_count['Average Homicide'] = crime_year_count.mean(axis=1)

crime_year_count['community_area'] = crime_year_count.index

crime_df = crime_year_count.melt('community_area')



/tmp/ipykernel_6126/942191158.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  crime_year_count['Average Homicide'] = crime_year_count.mean(axis=1)


In [207]:
grocery_stores = dct['grocery stores']

store_name = dct['grocery stores']['store_name']

grocery_stores['store_name'].str.contains('LIQUOR').value_counts()[True]

groc_count = dct['grocery stores'].groupby(by=['community_area']).size().to_frame(name='grocery stores count')

liquor_stores = grocery_stores[grocery_stores['store_name'].str.contains('LIQUOR')]

liquor_count = liquor_stores.groupby(by=['community_area']).size().to_frame(name='liquor stores count')

groceries_df = groc_count.merge(liquor_count, left_on = "community_area", right_on = "community_area")

groceries_df['Drunk Ratio'] = groceries_df['liquor stores count']/groceries_df['grocery stores count'] * 100

groceries_df['year'] = '2011'



In [ ]:
dct['hardship index'].columns.unique()
socio = dct['socioeconomic indicators']


Grocery stores: Status Done
Join on "community_area"
1. Goals
    a. columns: comm area, if possible comm area name, total groceries stores, num of liquor store, liquor/total gorcery stores ratio, year (??) - Sasha

Crime: Status Done
Join on "community_area"
1. Goals
    a. columns: comm area, if possible comm area name, homicide count, avg homicide rate (total homicide/no. of years) - Anthony

socioeconomic indicators
1. Goals
    a. columns: comm_area (make it a number), comma_area name, hardship index, year - Anthony